In [104]:
from zipfile import ZipFile
dataset='/content/sample_data/archive (2).zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('dataset has been extracted successfully')
#we just extracted the csv file from the zipfile named archive(2)

dataset has been extracted successfully


Importing the necessary libraries

In [120]:
import numpy as np    #for creating arrays(just a basic library fo ML)
import pandas as pd   #for feeding csv data into dataframe
import re             #for pattern matching or searching through an expression

#importing natural language libraries
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer   #To convert textual data into numerical data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk

Downloading the stopwords

In [121]:
nltk.download('stopwords')
print(stopwords.words('english'))
#These words are not required for processing more like they cant be classified into positive/neutral/negative

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


As the dataset is large by removing the stopwords we can make the dataset less complicated and the model would run more efficiently and these stopwords dont contribute a lot to training this model

In [122]:
#Getting data from the csv file and storing them in panda dataframe

chat_data=pd.read_csv('/content/chat_dataset.csv')
#default encoding UTF-8 is done here

chat_data.shape #(rows,column)

(584, 2)

In [123]:
chat_data.head()

,message,sentiment
0,I really enjoyed the movie,positive
1,The food was terrible,negative
2,I'm not sure how I feel about this,neutral
3,The service was excellent,positive
4,I had a bad experience,negative


In [124]:
chat_data['sentiment'].value_counts()
#Displays the number of texts that disttibuted within the sentiment category

,count
sentiment,
neutral,259
positive,178
negative,147


for better understanding I am gonna convert the targets as,
Positive = 1,
neutral = 0,
negative = -1

In [125]:
chat_data.replace({'sentiment':{'positive':1,'neutral':0,'negative':-1}},inplace=True)

<ipython-input-125-b432627c3426>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chat_data.replace({'sentiment':{'positive':1,'neutral':0,'negative':-1}},inplace=True)


In [126]:
#Displaying the modified sentiment distribution (feels better when i see 0s and 1s instead of positive or negative :) )
chat_data['sentiment'].value_counts()

,count
sentiment,
0,259
1,178
-1,147


**Stemming**

This process is done to convert a word into its root word so if the take the words like player/played/playing it will be converted into play. This method significantly reduces the complexity of out model and it can categorize the chat texts more easily

In [127]:
ps=PorterStemmer()

#stemming function
def stemming(text):
  modtext=re.sub('[^a-zA-Z]',' ',text)    #removes the letters that are not alphabet such as @, $, %, etc
  modtext=modtext.lower()                 #we dont want the model to fail just because it was case sensitive
  modtext=modtext.split()
  modtext=[ps.stem(word) for word in modtext if not word in stopwords.words('english')]
  modtext=' '.join(modtext)
  return modtext

In [128]:
chat_data['message']=chat_data['message'].apply(stemming) #thank god there is only 584 texts or else the system would have been cooked XD

In [129]:
chat_data.head()    #getting the stemmed content

,message,sentiment
0,realli enjoy movi,1
1,food terribl,-1
2,sure feel,0
3,servic excel,1
4,bad experi,-1


In [130]:
#spliting data and label
x=chat_data['message'].values
y=chat_data['sentiment'].values

In [131]:
#sending data to training sets

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
print(x_train)

['sure mood' 'great time' 'terribl product' 'servic restaur aw'
 'disappoint thing turn' 'sceneri beauti' 'servic excel'
 'got bad haircut wear hat week' 'hotel room spaciou clean'
 'wait come back' 'disappoint make mistak' 'feel inspir peopl around'
 'think go run' 'undecid' 'traffic light' 'phone okay'
 'love new job peopl work' 'grate beauti weather today' 'softwar ordinari'
 'amaz date last night' 'love new dress got sale' 'drink great' 'prefer'
 'love new music discov' 'sceneri averag' 'love cozi even good movi'
 'sure feel' 'hate feel like control life' 'look forward vacat'
 'game complet disast' 'softwar standard' 'hate feel alon isol'
 'studi final exam next week' 'custom servic great' 'speaker standard'
 'servic excel' 'typic product' 'tri decid two option' 'car expens'
 'grate littl thing life' 'love spend time pet'
 'grate health fit journey lost pound'
 'need buy new shoe work old one worn' 'sure think'
 'happi job love passion' 'feel well today' 'hate feel stress overwhelm

Now we are going to the main part of this model which is converting the text data into numerical data

In [132]:
v=TfidfVectorizer()
x_train=v.fit_transform(x_train)
x_test=v.transform(x_test)

#This block basically sets like a priority for repeated words in the texts
#This makes it easier for the model to categorize the texts into the sentimental categories

print(x_train)
#notice how the texts have been converted into numbers

  (0, 394)	0.4955279302630891
  (0, 245)	0.8685920045275453
  (1, 166)	0.7233001803662168
  (1, 411)	0.6905337421749919
  (2, 402)	0.7639205887284102
  (2, 312)	0.6453102619026287
  (3, 358)	0.4849546591301525
  (3, 339)	0.5389771601251313
  (3, 22)	0.6887108242589238
  (4, 90)	0.5697361871819706
  (4, 406)	0.4862736375872591
  (4, 424)	0.6625244345705338
  (5, 355)	0.7071067811865475
  (5, 28)	0.7071067811865475
  (6, 358)	0.6338372544624827
  (6, 121)	0.7734664406782377
  (7, 163)	0.2952850739274175
  (7, 24)	0.3647240951964652
  (7, 169)	0.4644700584847255
  (7, 451)	0.4352962344237548
  (7, 173)	0.4644700584847255
  (7, 454)	0.39854156287354814
  (8, 188)	0.37402704982881974
  (8, 343)	0.5061437424418614
  (8, 371)	0.5670291152198266
  :	:
  (458, 356)	0.42672141763753707
  (458, 132)	0.45532055215034906
  (458, 26)	0.45532055215034906
  (459, 269)	0.5679261841298967
  (459, 450)	0.5515829013031116
  (459, 49)	0.610914193950097
  (460, 195)	1.0
  (461, 411)	0.4876050126492427
  (46

Now we r gonna train our logistic regression model, so LETS GOOO!!!

In [133]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

**Model Prediction**

Accuracy score

In [136]:
x_pred=model.predict(x_train)
accuracy=accuracy_score(y_train,x_pred)
print('Training Accuracy=',accuracy)

Training Accuracy= 0.974304068522484


Now we are going to check how well our model works on the chat data

In [137]:
x_pred_test=model.predict(x_test)
accuracy=accuracy_score(y_test,x_pred_test)
print('Test Accuracy=',accuracy)

Test Accuracy= 0.7692307692307693


The test accuracy and the training accuracy are pretty much closer so we can say our model has did a good job.

Model accuracy:76.9%